# Download PDS data products

In the previous/first step we have queried PDS/ODE API for the data products over a region (Mawrth Vallis) of Mars.
In particular, we search for data from the MRO/CTX, MRO/HiRISE, and MEX/HRSC instruments.
The results -- i.e., footprints and associated metadata -- were written in GeoJSON file(s).

We will go now through the process of (1) downloading the/some data products, and (2) updating the metadata set we have for each image. Naturally, we are going to the (NPT) library to do it.

In [1]:
search_results = 'Mawrth_Vallis_ODE_products_intersect.geojson'

In [2]:
import npt

gjson = npt.utils.read_geojson(search_results)

In [3]:
import geopandas
import pandas
pandas.set_option('display.max_columns',100)

In [4]:
gdf = geopandas.read_file(search_results)

gdf_sample = gdf.groupby('inst').head(2).reset_index(drop=True)

gdf_sample

,id,mission,inst,type,Target_name,Footprints_cross_meridian,Map_scale,Center_latitude,Center_longitude,Easternmost_longitude,Westernmost_longitude,Minimum_latitude,Maximum_latitude,Emission_angle,Incidence_angle,Phase_angle,Solar_longitude,Observation_time,Product_creation_time,UTC_start_time,UTC_stop_time,image_url,label_url,browse_url,geometry
0,H1293_0000_ND3.JP2,MEX,HRSC,REFDR3,MARS,False,12.5,29.7761,341.8979,342.89,340.677,19.519,40.065,None,None,None,None,2005-01-19T19:06:48.344002,2016-03-04T20:31:26,2005-01-19T19:04:06.841000,2005-01-19T19:09:29.847000,https://pds-geosciences.wustl.edu/mex/mex-m-hr...,https://pds-geosciences.wustl.edu/mex/mex-m-hr...,https://pds-geosciences.wustl.edu/mex/mex-m-hr...,"POLYGON ((342.180 40.065, 342.190 39.882, 342...."
1,H2196_0001_ND3.JP2,MEX,HRSC,REFDR3,MARS,False,25,16.2676,344.0262,345.891,342.568,8.632,23.902,None,None,None,None,2005-09-29T15:28:07.719000,2016-03-14T11:38:04,2005-09-29T15:25:45.728001,2005-09-29T15:30:29.709999,https://pds-geosciences.wustl.edu/mex/mex-m-hr...,https://pds-geosciences.wustl.edu/mex/mex-m-hr...,https://pds-geosciences.wustl.edu/mex/mex-m-hr...,"POLYGON ((343.406 8.632, 343.399 8.787, 343.39..."
2,P01_001388_2034_XI_23N017W,MRO,CTX,EDR,MARS,False,5.66,23.2688,342.4958,342.8,342.19,22.8866,23.6504,5.85,48.88,43.11,134.36,2006-11-12T15:12:52.777000,2007-05-18T22:07:23,2006-11-12T15:12:46.050999,2006-11-12T15:12:59.504002,https://pds-imaging.jpl.nasa.gov/data/mro/mars...,None,None,"POLYGON ((342.710 23.650, 342.800 22.936, 342...."
3,P04_002641_2046_XN_24N017W,MRO,CTX,EDR,MARS,False,5.66,24.3758,342.2817,342.87,341.66,21.7462,27.0054,0.09,59.39,59.41,185.96,2007-02-18T05:52:00.318000,2007-08-09T21:43:46,2007-02-18T05:51:11.313000,2007-02-18T05:52:49.324001,https://pds-imaging.jpl.nasa.gov/data/mro/mars...,None,None,"POLYGON ((342.200 27.005, 342.870 21.806, 342...."
4,PSP_001388_2035_RED,MRO,HIRISE,RDRV11,MARS,False,0.25,23.2452,342.5015,342.563,342.439,23.161,23.3293,5.816564,48.900419,43.137054,134.35984,2006-11-12T15:12:52.248001,2009-09-22T22:45:09,2006-11-12T15:12:50.716999,2006-11-12T15:12:53.779999,https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...,https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...,https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/...,"POLYGON ((342.563 23.172, 342.459 23.161, 342...."
5,PSP_002641_2025_RED,MRO,HIRISE,RDRV11,MARS,False,0.25,22.2517,342.551,342.621,342.481,22.1027,22.4007,0.295027,58.737351,58.835998,185.953173,2007-02-18T05:51:20.181000,2009-11-11T10:51:58,2007-02-18T05:51:17.450001,2007-02-18T05:51:22.912001,https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...,https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...,https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/...,"POLYGON ((342.621 22.114, 342.518 22.103, 342...."


## Overview

During the whole of the downloading process is when we will fix eventual gaps in our metadata set.
For instance, have a look at the sample table above -- we have been carefull to select a couple of entries from the different datasets (_instruments_) so we can have a general view of the dataset at hand.

We notice the following points that should be fixed/adjusted at the end of this process:

- HRSC `id` is literally the product filename: remove the file extension from it to better define it as ID;
- Longitude coordinates are in the range [0:360]: shift it to [-180:180] to be compatible with typical geo-spatial databases and tools;
- HRSC entries have no values for `Emission_angle`, `Incidence_angle`, `Phase_angle`, `Solar_longitude`: recover them from image headers/Label files;
- CTX products have no separated Label files: write them from the Label information in the image headers.

## Split data

To keep things simple and under control, we want to download data products for each instrument separately.

In [5]:
gdf_sample_ctx = gdf_sample.query('inst == "CTX"')

gdf_sample_hirise = gdf_sample.query('inst == "HIRISE"')

gdf_sample_hrsc = gdf_sample.query('inst == "HRSC"')

## Convert (DF) tables to GeoJSON

GeoJSON (features) -- i.e., Python dictionaries -- is the data structure used in NPT download function(s).

In [6]:
import json

gjson_ctx = json.loads(gdf_sample_ctx.to_json())

gjson_hirise = json.loads(gdf_sample_hirise.to_json())

gjson_hrsc = json.loads(gdf_sample_hrsc.to_json())

## CTX

In [7]:
import npt

In [8]:
gjson_ctx_new = npt.download.from_geojson(gjson_ctx, basepath='data/ctx', progressbar=True)

function _is_downloaded()
*url = https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0008/data/P01_001388_2034_XI_23N017W.IMG
*filename = data/ctx/P01_001388_2034_XI_23N017W.IMG
-return True
PDS LABEL file 'data/ctx/P01_001388_2034_XI_23N017W.lbl' created.
function _is_downloaded()
*url = https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0057/data/P04_002641_2046_XN_24N017W.IMG
*filename = data/ctx/P04_002641_2046_XN_24N017W.IMG
-return True
PDS LABEL file 'data/ctx/P04_002641_2046_XN_24N017W.lbl' created.


In [9]:
gjson_hirise_new = npt.download.from_geojson(gjson_hirise, basepath='data/hirise', progressbar=True)

function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_001300_001399/PSP_001388_2035/PSP_001388_2035_RED.JP2
*filename = data/hirise/PSP_001388_2035_RED.JP2
-return False
--> Downloading file data/hirise/PSP_001388_2035_RED.JP2 ..


data/hirise/PSP_001388_2035_RED.JP2: 364637KB [23:26, 259.31KB/s]                                                                                                                              


--> File downloaded.
function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_001300_001399/PSP_001388_2035/PSP_001388_2035_RED.LBL
*filename = data/hirise/PSP_001388_2035_RED.LBL
-return False
--> Downloading file data/hirise/PSP_001388_2035_RED.LBL ..


data/hirise/PSP_001388_2035_RED.LBL: 8KB [00:00, 3076.98KB/s]                                                                                                                                  


--> File downloaded.
function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_002600_002699/PSP_002641_2025/PSP_002641_2025_RED.JP2
*filename = data/hirise/PSP_002641_2025_RED.JP2
-return False
--> Downloading file data/hirise/PSP_002641_2025_RED.JP2 ..


data/hirise/PSP_002641_2025_RED.JP2: 957981KB [54:03, 295.31KB/s]                                                                                                                              


--> File downloaded.
function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_002600_002699/PSP_002641_2025/PSP_002641_2025_RED.LBL
*filename = data/hirise/PSP_002641_2025_RED.LBL
-return False
--> Downloading file data/hirise/PSP_002641_2025_RED.LBL ..


data/hirise/PSP_002641_2025_RED.LBL: 8KB [00:00, 3210.34KB/s]                                                                                                                                  

--> File downloaded.


In [10]:
gjson_hrsc_new = npt.download.from_geojson(gjson_hrsc, basepath='data/hrsc', progressbar=True)

function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/1293/h1293_0000_nd3.jp2
*filename = data/hrsc/h1293_0000_nd3.jp2
-return False
--> Downloading file data/hrsc/h1293_0000_nd3.jp2 ..


data/hrsc/h1293_0000_nd3.jp2: 279607KB [09:58, 467.43KB/s]                                                                                                                                     


--> File downloaded.
function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/1293/h1293_0000_nd3.lbl
*filename = data/hrsc/h1293_0000_nd3.lbl
-return False
--> Downloading file data/hrsc/h1293_0000_nd3.lbl ..


data/hrsc/h1293_0000_nd3.lbl: 4KB [00:00, 640.91KB/s]                                                                                                                                          


--> File downloaded.
function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/2196/h2196_0001_nd3.jp2
*filename = data/hrsc/h2196_0001_nd3.jp2
-return False
--> Downloading file data/hrsc/h2196_0001_nd3.jp2 ..


data/hrsc/h2196_0001_nd3.jp2: 92268KB [04:05, 375.93KB/s]                                                                                                                                      


--> File downloaded.
function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/2196/h2196_0001_nd3.lbl
*filename = data/hrsc/h2196_0001_nd3.lbl
-return False
--> Downloading file data/hrsc/h2196_0001_nd3.lbl ..


data/hrsc/h2196_0001_nd3.lbl: 4KB [00:00, 1277.10KB/s]                                                                                                                                         

--> File downloaded.
